# Objetivo:


Aqui realizamos o fine-tuning de um modelo LLM usando técnicas eficientes em termos de memória, como o LoRA (Low-Rank Adaptation).

O objetivo é ajustar um modelo existente para que ele seja capaz de gerar respostas específicas baseadas nos dados de produtos que iremos passar!

In [1]:
#Conexão com o Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Instalação das bibliotecas necessárias
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install triton


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-san6iiwy/unsloth_1bf9a2c5a1424b19a7f71b46bf65649e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-san6iiwy/unsloth_1bf9a2c5a1424b19a7f71b46bf65649e
  Resolved https://github.com/unslothai/unsloth.git to commit d91d40a7b6b556f2d1fdd3e1e430f7a76a799627
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 31.2 MB/s eta 0:00:00
  


> Vamos carregar os modelos !

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# Comprimento máximo das sequências tokens de entrada para o modelo. 2048 é comum para grandes modelos de linguagem.
max_seq_length = 2048
# Tipo de dados usado para operações numéricas no modelo. Definido como 'None' aqui.
dtype = None
# Define que o modelo será carregado no modo de 4 bits, uma técnica para reduzir o uso de memória ao trabalhar com grandes modelos.
load_in_4bit = True

# Modelos que suportam o fine-tuning em 4 bits para uso eficiente de memória
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.




> Vamos carregar nosso modelo escolhido!



In [4]:

# Carregando o modelo e o tokenizador com suporte a 4 bits
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",  # Nome do modelo pré-treinado que será carregado.
    max_seq_length = max_seq_length,  # Comprimento máximo de tokens que o modelo pode processar.
    dtype = dtype,  # Tipo de dado numérico a ser utilizado.
    load_in_4bit = load_in_4bit,  # Define se o modelo será carregado no modo 4 bits para otimização de memória.
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]



> Vamos testar agora enviando uma descricão para ver se o modelo nos responde com o título do produto correto !



In [8]:
# Exemplo de inferência usando o modelo carregado
inputs = tokenizer(
    [
        "Encontre o seguinte produto. In a distant, timeless place, a mysterious prophet walks the sands."
    ],
    return_tensors="pt").to("cuda")  # Certifique-se de que está enviando os tensores para a GPU.

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Encontre o seguinte produto. In a distant, timeless place, a mysterious prophet walks the sands. The prophet is searching for the holy grail of the ancient world, the key to the secret of life. The prophet is searching for the holy grail of the ancient world, the key to the secret of life. The prophet is searching for the holy grail of the ancient world, the key to the secret of life. The prophet is searching for the holy grail of the ancient world, the key to the secret of life. The prophet is searching for the holy grail of the ancient world, the key to the secret of life. The prophet is searching for the holy grail of the ancient world, the key to the


> Não conseguimos chegar ao resultado que queríamos com esse input! O resultado deveria ser o título de produto "The Prophet" (Sem FineTuning não obtivemos o resultado correto).

